<a href="https://colab.research.google.com/github/sadhvikreddy/monte-carlo-simulation/blob/main/monte_carlo_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random

In [49]:
file = open('sample_input.txt', 'r')

input = pd.read_csv(file, delimiter='\t')

input.head()

,Golfer,Mean,Standard deviation
0,Sergio Garcia,68.998021,2.802211
1,Tiger Woods,69.351350,2.789151
2,Kenny Perry,69.914719,2.779165
3,Gonzalo Fernandez-Castano,70.004543,2.807307
4,Rory McIlroy,70.039146,2.815256


In [ ]:
input.columns = input.columns.str.strip()
transformed = pd.DataFrame(columns=['golfer', 'mean', 'standard_deviation'])
transformed['golfer'] = input['Golfer']
transformed['mean'] = pd.to_numeric(input['Mean'])
transformed['standard_deviation'] = pd.to_numeric(input['Standard deviation'])

transformed['position'] = pd.factorize(transformed['mean'])[0] + 1

transformed.head(25)

,golfer,mean,standard_deviation,position
0,Sergio Garcia,68.998021,2.802211,1
1,Tiger Woods,69.351350,2.789151,2
2,Kenny Perry,69.914719,2.779165,3
3,Gonzalo Fernandez-Castano,70.004543,2.807307,4
4,Rory McIlroy,70.039146,2.815256,5
5,John Cook,70.073148,2.748841,6
6,Jaco Van Zyl,70.089262,2.784261,7
7,Brandt Snedeker,70.179877,2.771891,8
8,Francesco Molinari,70.216814,2.748345,9
9,Matteo Manassero,70.220108,2.773465,10


In [ ]:
# Generic function for win friction
def calculate_column_friction(column, position, total_in_position):
  return 1 / total_in_position if column <= position else 0

# caluclate win friction based on the requirement
def caluclate_win_frictions(df):
  df['win_friction'] = df['position'].apply(lambda c: calculate_column_friction(c, 1, df['position'].eq(c).sum()))
  df['top_5_friction'] = df['position'].apply(lambda c: calculate_column_friction(c, 5, df['position'].eq(c).sum()))
  return df

In [35]:
# for one tournament simulation, we generate the values for 4 rounds and add them.

def generateOneSimulation():
  one_simulation = pd.DataFrame(columns=['golfer', 'generated_score'])

  one_simulation['golfer'] = transformed['golfer']
  one_simulation['generated_score_1'] = random.normalvariate(transformed['mean'], transformed['standard_deviation']).astype(int)
  one_simulation['generated_score_2'] = random.normalvariate(transformed['mean'], transformed['standard_deviation']).astype(int)
  one_simulation['generated_score_3'] = random.normalvariate(transformed['mean'], transformed['standard_deviation']).astype(int)
  one_simulation['generated_score_4'] = random.normalvariate(transformed['mean'], transformed['standard_deviation']).astype(int)
  one_simulation['generated_score'] = (one_simulation['generated_score_1'] + one_simulation['generated_score_2'] + one_simulation['generated_score_3'] + one_simulation['generated_score_4'])
  one_simulation['position'] = pd.factorize(one_simulation['generated_score'])[0] + 1

  one_simulation_with_fractions = caluclate_win_frictions(one_simulation)
  return one_simulation_with_fractions

In [36]:
def run_simulations(n):
 simulations = []
 for i in range(n):
  oneSim = generateOneSimulation()
  simulations.append({'table': oneSim})


 for_avg = pd.concat([a['table'] for a in simulations])
 averages = (for_avg.groupby('golfer')[['win_friction','top_5_friction']].mean())

 print(averages.sort_values(by=['win_friction', 'top_5_friction'], ascending=[False, False]))

In [42]:
run_simulations(10)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                        0.8        0.800000
Tiger Woods                          0.2        0.800000
Kenny Perry                          0.0        0.327837
Charl Schwartzel                     0.0        0.230476
Darren Fichardt                      0.0        0.190476
Gonzalo Fernandez-Castano            0.0        0.167837
Jay Haas                             0.0        0.146726
Jaco Van Zyl                         0.0        0.126171
Rory McIlroy                         0.0        0.126171
Brandt Snedeker                      0.0        0.113313
Francesco Molinari                   0.0        0.104980
Matteo Manassero                     0.0        0.104980
Duffy Waldorf                        0.0        0.103393
John Cook                            0.0        0.099980
Jim Furyk                            0.0        0.093869
Justin Rose                    

In [38]:
run_simulations(100)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                      0.925        0.925000
Tiger Woods                        0.075        0.890000
Kenny Perry                        0.000        0.428608
Gonzalo Fernandez-Castano          0.000        0.241543
Rory McIlroy                       0.000        0.209816
John Cook                          0.000        0.197856
Jaco Van Zyl                       0.000        0.176432
Jay Haas                           0.000        0.157007
Brandt Snedeker                    0.000        0.155789
Matteo Manassero                   0.000        0.149766
Francesco Molinari                 0.000        0.143142
Darren Fichardt                    0.000        0.131280
Rocco Mediate                      0.000        0.108782
Luke Donald                        0.000        0.105417
Charl Schwartzel                   0.000        0.103225
Phil Mickelson                 

In [43]:
run_simulations(1000)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                     0.9175        0.917500
Tiger Woods                       0.0825        0.894750
Kenny Perry                       0.0000        0.445808
Gonzalo Fernandez-Castano         0.0000        0.232030
Rory McIlroy                      0.0000        0.193806
John Cook                         0.0000        0.186150
Jaco Van Zyl                      0.0000        0.172691
Jay Haas                          0.0000        0.157166
Brandt Snedeker                   0.0000        0.143254
Darren Fichardt                   0.0000        0.129948
Charl Schwartzel                  0.0000        0.119834
Francesco Molinari                0.0000        0.118451
Matteo Manassero                  0.0000        0.114264
Duffy Waldorf                     0.0000        0.113530
Rocco Mediate                     0.0000        0.106033
Phil Mickelson                 

In [44]:
run_simulations(5000)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                     0.9141        0.914100
Tiger Woods                       0.0859        0.893458
Kenny Perry                       0.0000        0.448344
Gonzalo Fernandez-Castano         0.0000        0.230154
Rory McIlroy                      0.0000        0.194455
John Cook                         0.0000        0.178850
Jaco Van Zyl                      0.0000        0.167725
Jay Haas                          0.0000        0.157651
Brandt Snedeker                   0.0000        0.141946
Darren Fichardt                   0.0000        0.134207
Francesco Molinari                0.0000        0.117888
Charl Schwartzel                  0.0000        0.116991
Duffy Waldorf                     0.0000        0.113426
Matteo Manassero                  0.0000        0.112633
Rocco Mediate                     0.0000        0.105194
South Africa                   

In [45]:
run_simulations(5000)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                   0.915067        0.915067
Tiger Woods                     0.084867        0.891855
Kenny Perry                     0.000067        0.445469
Gonzalo Fernandez-Castano       0.000000        0.231003
Rory McIlroy                    0.000000        0.194597
John Cook                       0.000000        0.183908
Jaco Van Zyl                    0.000000        0.173171
Jay Haas                        0.000000        0.161441
Brandt Snedeker                 0.000000        0.143558
Darren Fichardt                 0.000000        0.128371
Francesco Molinari              0.000000        0.118732
Matteo Manassero                0.000000        0.112675
Charl Schwartzel                0.000000        0.112614
Duffy Waldorf                   0.000000        0.109507
Rocco Mediate                   0.000000        0.107739
South Africa                   

In [46]:
run_simulations(5000)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                     0.9164        0.916400
Tiger Woods                       0.0836        0.897030
Kenny Perry                       0.0000        0.443167
Gonzalo Fernandez-Castano         0.0000        0.230394
Rory McIlroy                      0.0000        0.194190
John Cook                         0.0000        0.178864
Jaco Van Zyl                      0.0000        0.170428
Jay Haas                          0.0000        0.162047
Brandt Snedeker                   0.0000        0.143878
Darren Fichardt                   0.0000        0.131592
Francesco Molinari                0.0000        0.120105
Charl Schwartzel                  0.0000        0.115692
Matteo Manassero                  0.0000        0.115170
Duffy Waldorf                     0.0000        0.110824
Rocco Mediate                     0.0000        0.106935
South Africa                   

In [47]:
run_simulations(5000)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                      0.915        0.915000
Tiger Woods                        0.085        0.892474
Kenny Perry                        0.000        0.444752
Gonzalo Fernandez-Castano          0.000        0.231257
Rory McIlroy                       0.000        0.196658
John Cook                          0.000        0.183518
Jaco Van Zyl                       0.000        0.170525
Jay Haas                           0.000        0.161247
Brandt Snedeker                    0.000        0.145382
Darren Fichardt                    0.000        0.133460
Francesco Molinari                 0.000        0.119034
Charl Schwartzel                   0.000        0.117219
Matteo Manassero                   0.000        0.116065
Duffy Waldorf                      0.000        0.111828
Rocco Mediate                      0.000        0.109412
South Africa                   

In [48]:
run_simulations(5000)

                            win_friction  top_5_friction
golfer                                                  
Sergio Garcia                     0.9151        0.915100
Tiger Woods                       0.0849        0.895239
Kenny Perry                       0.0000        0.446830
Gonzalo Fernandez-Castano         0.0000        0.231595
Rory McIlroy                      0.0000        0.196135
John Cook                         0.0000        0.183081
Jaco Van Zyl                      0.0000        0.168772
Jay Haas                          0.0000        0.164863
Brandt Snedeker                   0.0000        0.144577
Darren Fichardt                   0.0000        0.135704
Francesco Molinari                0.0000        0.118742
Charl Schwartzel                  0.0000        0.115926
Matteo Manassero                  0.0000        0.113810
Duffy Waldorf                     0.0000        0.112820
Rocco Mediate                     0.0000        0.107475
South Africa                   

With 5000 iterations, the results look stable.